In [1]:
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("LCEL-Advanced")

LangSmith 추적을 시작합니다.
[프로젝트명]
LCEL-Advanced


# LLM API Error 에 대처 방법

In [2]:
from langchain_anthropic import ChatAnthropic
from langchain_openai import ChatOpenAI

In [3]:
from unittest.mock import patch

import httpx
from openai import RateLimitError

request = httpx.Request("GET", "/")  # GET 요청을 생성합니다.
response = httpx.Response(
    200, request=request
)  # 200 상태 코드와 함께 응답을 생성합니다.
# "rate limit" 메시지와 응답 및 빈 본문을 포함하는 RateLimitError를 생성합니다.
error = RateLimitError("rate limit", response=response, body="")

In [4]:
# OpenAI의 ChatOpenAI 모델을 사용하여 openai_llm 객체를 생성합니다.
# max_retries를 0으로 설정하여 속도 제한 등으로 인한 재시도를 방지합니다.
openai_llm = ChatOpenAI(max_retries=0)

# Anthropic의 ChatAnthropic 모델을 사용하여 anthropic_llm 객체를 생성합니다.
anthropic_llm = ChatAnthropic(model="claude-3-opus-20240229")

# openai_llm을 기본으로 사용하고, 실패 시 anthropic_llm을 대체로 사용하도록 설정합니다.
llm = openai_llm.with_fallbacks([anthropic_llm])

In [5]:
# OpenAI LLM을 먼저 사용하여 오류가 발생하는 것을 보여줍니다.
with patch("openai.resources.chat.completions.Completions.create", side_effect=error):
    try:
        # "닭이 길을 건넌 이유는 무엇일까요?"라는 질문을 OpenAI LLM에 전달합니다.
        print(openai_llm.invoke("Why did the chicken cross the road?"))
    except RateLimitError:
        # 오류가 발생하면 오류를 출력합니다.
        print("에러 발생")

에러 발생


In [6]:
# OpenAI API 호출 시 에러가 발생하는 경우 Anthropic 으로 대체하는 코드
with patch("openai.resources.chat.completions.Completions.create", side_effect=error):
    try:
        # "대한민국의 수도는 어디야?"라는 질문을 언어 모델에 전달하여 응답을 출력합니다.
        print(llm.invoke("대한민국의 수도는 어디야?"))
    except RateLimitError:
        # RateLimitError가 발생하면 "에러 발생"를 출력합니다.
        print("에러 발생")

에러 발생


# fallback 에 여러 모델을 순차적으로 지정

In [7]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# 프롬프트 생성
prompt_template = (
    "질문에 짧고 간결하게 답변해 주세요.\n\nQuestion:\n{question}\n\nAnswer:"
)
prompt = PromptTemplate.from_template(prompt_template)

In [8]:
# 여기서는 쉽게 오류를 발생시킬 수 있는 잘못된 모델 이름을 사용하여 체인을 생성할 것입니다.
chat_model = ChatOpenAI(model_name="gpt-fake")
bad_chain = prompt | chat_model

In [9]:
# fallback 체인을 생성합니다.
fallback_chain1 = prompt | ChatOpenAI(model="gpt-3.6-turbo") # 오류
fallback_chain2 = prompt | ChatOpenAI(model="gpt-3.5-turbo") # 정상
fallback_chain3 = prompt | ChatOpenAI(model="gpt-4-turbo-preview") # 정상

In [10]:
# 두 개의 체인을 결합하여 최종 체인을 생성합니다.
chain = bad_chain.with_fallbacks(
    [fallback_chain1, fallback_chain2, fallback_chain3])
# 생성된 체인을 호출하여 입력값을 전달합니다.
chain.invoke({"question": "대한민국의 수도는 어디야?"})

AIMessage(content='서울입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 46, 'total_tokens': 52, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BMTBpyBNcPE3ysc5Bhnav68u6ah5H', 'finish_reason': 'stop', 'logprobs': None}, id='run-8cbb36b3-2f84-42d0-8f28-f427f15ac142-0', usage_metadata={'input_tokens': 46, 'output_tokens': 6, 'total_tokens': 52, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})